In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier


In [2]:
# Load the Titanic dataset
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Combine train and test datasets for consistent preprocessing
combine = [train_df, test_df]

# Preview data
train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Use KNN Imputation for missing Age values based on key features
age_features = ["Pclass", "SibSp", "Parch", "Fare"]
knn_imputer = KNNImputer(n_neighbors=5)
train_df["Age"] = knn_imputer.fit_transform(train_df[age_features])
test_df["Age"] = knn_imputer.transform(test_df[age_features])

# Fill missing Embarked values with mode
train_df["Embarked"].fillna(train_df["Embarked"].mode()[0], inplace=True)
test_df["Embarked"].fillna(test_df["Embarked"].mode()[0], inplace=True)

# Fill missing Fare values with median
test_df["Fare"].fillna(test_df["Fare"].median(), inplace=True)


C:\Users\Manali Parikh\AppData\Local\Temp\ipykernel_22428\2961309010.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["Embarked"].fillna(train_df["Embarked"].mode()[0], inplace=True)
C:\Users\Manali Parikh\AppData\Local\Temp\ipykernel_22428\2961309010.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [4]:
# Create "Family Size" and "IsAlone" features
for dataset in combine:
    dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1
    dataset["IsAlone"] = (dataset["FamilySize"] == 1).astype(int)

# Extract Title from Name
for dataset in combine:
    dataset["Title"] = dataset["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)

# Replace rare titles with "Rare"
title_mapping = {
    "Capt": "Rare", "Col": "Rare", "Major": "Rare", "Dr": "Rare", "Rev": "Rare",
    "Jonkheer": "Rare", "Dona": "Rare", "Countess": "Rare", "Don": "Rare",
    "Sir": "Rare", "Lady": "Rare", "Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"
}
for dataset in combine:
    dataset["Title"] = dataset["Title"].map(title_mapping).fillna("Other")

# Extract Deck from Cabin
for dataset in combine:
    dataset["Deck"] = dataset["Cabin"].astype(str).str[0].replace("n", "U")  # Replace missing cabins


In [5]:
# One-hot encode categorical features
categorical_features = ["Embarked", "Sex", "Title", "Deck"]
train_df = pd.get_dummies(train_df, columns=categorical_features, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical_features, drop_first=True)


In [6]:
from sklearn.preprocessing import MinMaxScaler

# Use MinMaxScaler (0 to 1) instead of StandardScaler
minmax_scaler = MinMaxScaler()
train_df[["Age", "Fare", "FamilySize"]] = minmax_scaler.fit_transform(train_df[["Age", "Fare", "FamilySize"]])
test_df[["Age", "Fare", "FamilySize"]] = minmax_scaler.transform(test_df[["Age", "Fare", "FamilySize"]])


In [7]:
# Drop unnecessary columns before training
drop_columns = ["Name", "Ticket", "Cabin", "PassengerId"]
X = train_df.drop(["Survived"] + drop_columns, axis=1)
y = train_df["Survived"]

# Split dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Support Vector Machines": SVC(),
    "Naive Bayes": GaussianNB(),
    "Perceptron": Perceptron(alpha=0.001, class_weight="balanced", max_iter=1000, tol=1e-3),
    "Linear SVC": LinearSVC(),
    "Stochastic Gradient Descent": SGDClassifier(loss="log_loss", penalty="l2", alpha=0.0001, max_iter=2000, tol=1e-4, learning_rate="adaptive", eta0=0.01),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=3),
}

In [ ]:
# Train models and store accuracy results
accuracy_results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    accuracy = round(model.score(X_train, y_train) * 100, 2)
    accuracy_results[model_name] = accuracy

# Display results
for model, acc in accuracy_results.items():
    print(f"{model}: {acc}%")

Logistic Regression: 80.62%
Support Vector Machines: 81.18%
Naive Bayes: 70.79%
Perceptron: 79.49%
Linear SVC: 80.9%
Stochastic Gradient Descent: 80.9%
Decision Tree: 93.68%
Random Forest: 93.68%
K-Nearest Neighbors: 86.24%
